In [1]:
import datetime as dt

import pandas as pd

from private import DartPrivate
from etl_tool import DartFssBroker, DartFssExtractor, DartCorpDfPreproc
from etl_tool import DartApiBroker, DartApiExtractor, DartFundamentalDfPreProc

In [2]:
dart_fss_broker =  DartFssBroker(DartPrivate.api_key)
dart_fss_extractor = DartFssExtractor(dart_fss_broker)

dart_api_broker =  DartApiBroker(DartPrivate.api_key)
dart_api_extractor = DartApiExtractor(dart_api_broker)

In [4]:
corp_df = dart_fss_extractor.extract_corp_df()

In [6]:
public_df = DartCorpDfPreproc.filter_public(corp_df)
public_df = public_df[public_df["corp_cls"].isin(["Y", "K"])]

In [8]:
today = dt.date.today()

bsns_years = [str(today.year - i) for i in range(6)]
reprt_codes = ["11013", "11012", "11014", "11011"]
corp_codes = list(set(public_df["corp_code"]))

In [9]:
fundamental_df_list = list()
for bsns_year in bsns_years:
    for reprt_code in reprt_codes:
        _fundamental_df = dart_api_extractor.extract_multi_fundamental_df(
            corp_codes=corp_codes, bsns_year=bsns_year, reprt_code=reprt_code
        )
        fundamental_df_list.append(_fundamental_df)

In [10]:
fundamental_df = pd.concat(fundamental_df_list, axis=0)

In [23]:
raw_fundamental_df = fundamental_df.copy()

In [24]:
raw_fundamental_df["bfefrmtrm_amount"] = raw_fundamental_df["bfefrmtrm_amount"].fillna("-")
raw_fundamental_df["frmtrm_amount"] = raw_fundamental_df["frmtrm_amount"].fillna("-")
raw_fundamental_df["thstrm_amount"] = raw_fundamental_df["thstrm_amount"].fillna("-")

In [25]:
dart_fundamental_df_pre_proc = DartFundamentalDfPreProc(raw_fundamental_df)

In [26]:
cfs_fundamental_df = dart_fundamental_df_pre_proc('CFS')
ofs_fundamental_df = dart_fundamental_df_pre_proc('OFS')

In [28]:
cfs_fundamental_df.to_csv("./data/dart/cfs_fundamental_df.csv")
ofs_fundamental_df.to_csv("./data/dart/ofs_fundamental_df.csv")